# Introduction
State notebook purpose here

### Imports
Import libraries and write settings here.

In [1]:
# Notebooks specific imports
from IPython import get_ipython
ipython = get_ipython()
#Expand notebook display

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload
%autoreload 2
from tqdm.notebook import tqdm

# Basic useful imports
import re
import time
import yaml
from pprint import pprint
from pathlib import Path
import h5py
import warnings

# Data manipulation
import numpy as np
from scipy.special import erf
from scipy.integrate import quad
import scipy.stats as stats
from scipy.signal import savgol_filter
from scipy.spatial import ConvexHull

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
from matplotlib.patches import (Circle, RegularPolygon, FancyArrowPatch, ArrowStyle)
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator, NullFormatter)
import matplotlib.colors as mcolors

# Clustering stuff
from sklearn.cluster import MeanShift, estimate_bandwidth, DBSCAN, OPTICS
from itertools import cycle
# plt.cm.tab20.colors

# From alens_analysis.py
import alens_analysis as aa
import alens_analysis.chromatin as aac
import alens_analysis.chromatin.chrom_analysis as ca
import alens_analysis.chromatin.chrom_condensate_analysis as cca
import alens_analysis.chromatin.chrom_graph_funcs as cgf
from alens_analysis import cluster_analysis as cla

from alens_analysis.colormaps import register_cmaps

# Locations
ws_path = Path('/home/alamson/DATA/Chromatin/')
mnt_path = Path.home() / 'projects/DATA/Chromatin/'
ceph_path = Path.home() / 'ceph/DATA/Chromatin/'

In [2]:
# Consistent settings
ss_ind = 1
end_ind = None
start_bead = 0
end_bead = None

### Style settings

In [3]:
graph_sty = {
    "axes.titlesize": 20,
    "axes.labelsize": 24,
    "lines.linewidth": 2,
    "lines.markersize": 2,
    "xtick.labelsize": 24,
    "ytick.labelsize": 24,
    "font.size": 20,
    "font.sans-serif": 'Helvetica',
    "text.usetex": False,
    'mathtext.fontset': 'cm',
}
plt.style.use(graph_sty)

colors = cycle(mcolors.XKCD_COLORS.keys())
# mcolors.__dict__
# colors = cycle("bgrcmykbgrcmykbgrcmykbgrcmyk")

register_cmaps()
#plt.rcParams['image.cmap'] = 'emct8'
#plt.rcParams['image.cmap'] = 'warm'
plt.rcParams['image.cmap'] = 'YlOrRd'
#plt.rcParams['image.cmap'] = 'twilight'
#plt.rcParams['image.cmap'] = 'coolwarm'
#plt.rcParams['image.cmap'] = 'RdYlBu_r'
        

In [4]:
for i, c in zip([1,2,3], mcolors.XKCD_COLORS.keys()):
    print(i, c)
    

1 xkcd:cloudy blue
2 xkcd:dark pastel green
3 xkcd:dust


## Figure 2a (number of beads in condensates)

In [5]:
def plot_confidence_int(ax, mean, std, color='b', ci=.95):
    degrees_freedom = avg_num_clust_beads_arr.shape[0] - 1
    confidence_interval = stats.t.ppf((1 + ci) / 2., degrees_freedom) * std_dev / np.sqrt(avg_num_clust_beads_arr.shape[0])

    _ = ax.plot(time_arr, mean, label='Mean', color=color)
    _ = ax.fill_between(time_arr, mean - confidence_interval, mean + confidence_interval, color=color, alpha=.1)

In [6]:
data_path_list = [ceph_path / "DynCondPaper/23-10-20_aLc1_slice4.5.13_line1600_Pin3-9um_2xKe3-300_ks100/simulations/Ke30_Pin5.0um/", 
                  ceph_path / "DynCondPaper/23-10-20_aLc1_slice4.5.13_line1600_Pin3-9um_2xKe3-300_ks100/simulations/Ke30_Pin3.0um/",
                  ceph_path / "DynCondPaper/23-10-20_aLc1_slice4.5.13_line1600_Pin3-9um_2xKe3-300_ks100/simulations/Ke30_Pin9.0um/"]
# [x] Baseline (2x Ke 30, pin 5um)
# [x] short_separation (2x Ke 30, pin 3um)
# [x] large separation (2x Ke 30 pin 9um
# low Ke (2x Ke 3, pin 5um)
# high Ke (2x Ke 300, pin 5um)

part_min=40

try:
    fig, axarr = plt.subplots(1, 1, figsize=(10, 6))
    for data_path, color, label in zip(data_path_list, mcolors.XKCD_COLORS.keys(), ['Baseline', 'Short separation', 'Large separation']):
        print(data_path)
        sd_h5_file_lst = [h5p for h5p in data_path.glob('s*/analysis/raw_data.h5')]
        with h5py.File(sd_h5_file_lst[0], 'r') as h5d:
            time_arr = h5d['time'][ss_ind:end_ind]
            nbeads = h5d['raw_data']['sylinders'][start_bead:end_bead, 0, 0].shape[0]

        avg_num_clust_beads_arr = np.zeros((len(sd_h5_file_lst), time_arr.size))
        for ih, h5d in enumerate(sd_h5_file_lst):
            h5_clust_file = h5d.parent / 'cluster_analysis.h5'
            with h5py.File(h5_clust_file, 'r') as h5_data:
                cluster_grp = h5_data['clusters']
                # time_arr = h5_data['time'][...]
                time_grp_list = sorted(cluster_grp.values(), key=lambda x: x.attrs['time'])
                clusters = []
                for tg in time_grp_list:
                    clusters += [[cla.Cluster(h5_data = c) for c in tg.values()]]

            # num_clusters_list = np.zeros(len(clusters))
            # avg_num_cluster_beads_list = np.zeros(len(clusters))
            assert len(clusters) == time_arr.size

            bead_ind_arr = np.zeros((nbeads, time_arr.size))
            one_mask = np.ones(nbeads)
            for c, clust_grp in enumerate(clusters):
                # Secondary thresholding
                clust_grp = [clust for clust in clust_grp if len(clust.part_ids) > part_min]
                # num_clusters_list[c] += [len(clust_grp)]
                num_beads = 0
                for i, clust in enumerate(clust_grp):
                    num_beads += len(clust.part_ids)
                    bead_ind_arr[clust.part_ids, c] += one_mask[clust.part_ids]

                avg_num_clust_beads_arr[ih, c] += num_beads

        mean = avg_num_clust_beads_arr.mean(axis=0)
        std_dev = avg_num_clust_beads_arr.std(axis=0)
        plot_confidence_int(axarr, mean, std_dev, color=color)

    _ = axarr.set_ylabel('Number of beads in clusters')
    _ = axarr.set_xlabel('Time (sec)')
    _ = axarr.legend(loc='center left', bbox_to_anchor=(1.05, .5))
        

except:
    raise
finally:
    pass
    # for h5d in sd_h5_data_lst:
    #     h5d.close()

/mnt/home/alamson/ceph/DATA/Chromatin/DynCondPaper/23-10-20_aLc1_slice4.5.13_line1600_Pin3-9um_2xKe3-300_ks100/simulations/Ke30_Pin5.0um
